Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "" # put your full name here
COLLABORATORS = [] # list anyone you collaborated with on this workbook

---

## Lab 9: Classification & CART

Welcome to the ninth lab of the semester!

In this lab, we'll do our first classification problem. With classification, we are working to predict response variables that are qualitative (i.e. categorical) rather than quantitative.

Some real-life examples of classification, from ISLR, are:
1. A person arrives at the emergency room with a set of symptoms that could possibly be attributed to one of three medical conditions. Which of the three conditions does the individual have?
1. An online banking service must be able to determine whether or not a transaction being performed on the site is fraudulent, on the basis of the user’s IP address, past transaction history, and so forth.
1. On the basis of DNA sequence data for a number of patients with and without a given disease, a biologist would like to figure out which DNA mutations are deleterious (disease-causing) and which are not.

Logistic regression is one method for solving classification problems. KNN and decision trees are two others.

### Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore', FutureWarning) # Seaborn triggers warnings in scipy
%matplotlib inline
from matplotlib.colors import ListedColormap

In [ ]:
# Configure nice plotting defaults - (this must be done in a cell separate from %matplotlib call)
plt.style.use('seaborn-v0_8')
sns.set_context('talk', font_scale=1)
plt.rcParams['figure.figsize'] = (10, 6)

----
### Logistic Regression

Let's start with the simple example of a modelling problem where the categorical variable can take on two values. For example, take a look at example #2 above - a bank needs to determine whether a transaction is fraudulent, so it is trying to model a situation with two potential values of the response variable: either the transaction was fraudulent (we'll give this category a value of Y = 1) or it was not (we'll give this category a value of Y = 0).

Since Y isn't a *continuous* variable - it can only take on a value of 0 or a value of 1, and these values don't represent any particular ordering - we use logistic regression, which models the *probability* that Y = 0 or Y = 1. We can write this probability as:

$ Pr(Y = 1|X)$ 

The expression above, in words, means "the probability that Y = 1 given X". In our case, Y = 1 means the transaction was fradulent, and X is the matrix of predictor variables (transaction history, IP address, etc) that we're using to predict Y. This probability expression can be written using the following shorthand:

$ Pr(Y = 1|X) = p(X)$ 

We *could* model this probability using a simple linear regression model, that would look something like this:

$p(X) = \beta_0 + \beta_1X_1$

As we'll see throughout the lab, however, simple linear regression doesn't really give us meaningful results when it comes to classification problems. Instead, logistic regression proposes a model that uses the logistic function, which looks something like this:

$p(X) = \frac{e^{\beta_0 + \beta_1X_1}}{1 + e^{\beta_0 + \beta_1X_1}}$

The two model forms above (linear and logistic regression) have only one feature $X_1$, but can be easily extended to multiple features. The key to the logistic function is that it can only evaluate to values between 0 and 1, so it will always represent a probability.

Another important and related parameter is the **odds**. In statistics, the odds are defined as the probability of success over the probability of failure. If something is 50% likely to succeed, then its odds are 1; there is an equal probability of success to failure. If it's 80% likely to succed, its odds are 0.8/(1-0.8) = 4 - there is a 4 to 1 chance of success to failure. The odds of $p(X)$ can be written as:

$\frac{p(X)}{1-p(X)}$

The odds of $p(X)$ can take any value between 0 and infinity. The logistic function can be re-written in the following way:

$log\left(\frac{p(X)}{1-p(X)}\right) = \beta_0 + \beta_1X_1$

This should look super familiar! It looks *almost* like a linear regression. Basically, we can represent the log-odds (also called the logit) as a function that is linear in $X_1$. It's important to be careful with interpretation here, though - even though the relationship between the logit and $X$ is linear, the relationship between $p(X)$ and $X$ is not. A unit increase in $X_1$ in the formula above represents an increase equal to $\beta_1$ in the *log-odds* of $p(X)$, which is the same as multiplying the odds by $e^{\beta_1}$. The amount that $p(X)$ changes when $X$ changes depends on the current value of $X$. However, there is a simple interpretation to the signs of the coefficients - if $\beta_1$ is positive, that means that an increase in $X_1$ leads to an increased probability of $p(X)$.

The basic intuition for how to choose the $\beta$ coefficients for logistic regression is similar to linear regression. With linear regression, we want to choose coefficients that make the prediction as close as possible to the observation $y$. With logistic regression, we want to choose coefficients that make the predicted probability (of, for example, a fraudulent transaction) as close as possible to the actual category of the transaction (1 for a fraudulent transaction, 0 for a non-fraudulent transaction). Logistic regression software functions work by maximizing something called the likelihood function, much like linear regression functions work by minimizing the mean squared error.

---
Now that we've gone through the basics, we can load our data and try implementing logistic regression.

In this section, we'll work with a modified version of the [ozone level detection dataset](https://archive.ics.uci.edu/ml/datasets/ozone+level+detection) from the UCI Machine Learning Repository, which uses temperature, wind speed, pressure, and other features to decide if a specific day was in fact a normal day or a high ground level ozone day. 

Run the cell below to load ozone.csv into dataframe `df`.

In [ ]:
# run this cell
df = pd.read_csv('data/ozone.csv')
df.head()

Looking at the columns, we can infer that columns WSR0, WSR1, etc. are the hourly measurements for the wind speed, and the second-to-last column, `Class`, is the variable we want to predict. 0 is a  normal day and 1 is an ozone day. If you want more information on the features, you can read up on the description of the data [here](https://archive.ics.uci.edu/ml/datasets/ozone+level+detection). 

**Question 1.1 (1pt)** Create a scatter plot showing `Class` versus `WSR_PK`. 

In [ ]:
# YOUR CODE HERE
plt.scatter(df.WSR_PK, df.Class)
plt.xlabel(...)
plt.ylabel(...);

**Question 1.2 (1pt)** Run the cell below to fit a linear regression to the data shown above. Based on the plot and your understanding of the data, why would linear regression be a poor choice for a predictive model in this case?

In [ ]:
# run this cell
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(df[["WSR_PK"]], df[["Class"]])

plt.scatter(df.WSR_PK, df.Class)
plt.plot(df.WSR_PK, lm.predict(df[["WSR_PK"]]), c = "y")
plt.xlabel("Wind speed")
plt.ylabel("Ozone day status")
plt.title("Linear regression fit to ozone status");

*YOUR ANSWER HERE*

Let's try building a predictive model using classification! We'll build a logistic regression model with statsmodels. We'll use the [`Logit()` function](https://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.Logit.html) function.

As an aside: we could also use scikit-learn's `LogisticRegression()` function, but scikit-learn performs logistic regression using a penalty by default, while statsmodel uses maximum likelihood estimation to choose coefficients. Both are useful depending on how you want to solve for your coefficients!

**Question 1.3 (1pt)** Fit a logistic regression on the ozone dataset, using "Class" as the response variable and `WSR_PK` as the predictor. The code below gets you started. You may remember that statsmodels models require the `X` input to contain a column of ones to represent the intercept. Conveniently, the last column of our dataframe `df` is a column of ones called `intercept`. (If it didn't, we could use `np.ones()` to create a column of ones). Using the code below as a starting point, run a logistic regression that predicts `Class` given `WSR_PK` and `intercept`, and then print the parameters.

If you can't remember the general process for how to fit models with statsmodels, look back at Lab 6.

In [ ]:
import statsmodels.api as sm

In [ ]:
# Replace ellipses with your code
y = df[...]
X = df[[...]]

logit = sm.Logit(...)
logit_fit = ...
logit_fit.params

Let's use these parameters (aka weights or coefficients) shown above to plot predicted probabilities. To do that, we have to be able to calculate probability from the weights. Recall the relationship:
$$p(X) = \frac{e^{\beta_0 + \beta_1X}}{1+e^{\beta_0 + \beta_1X}}$$

**Question 1.4 (2pts)** Write a function `prob_logit()` that returns a vector of probabilities p(X) given a vector of $\beta$ values and a matrix of $X$ values. The function `np.exp()` will be helpful here.

In [ ]:
def prob_logit(params, X):
    """
    Objective: This function calculates the probability p(X) given a set of weights (params) and a matrix of X values.
    Inputs:
        params: a vector of weights from a logistic regression fit
        X: a matrix of predictor values with a number of columns equal to number of elements in params
    Output:
        p: a vector of probabilities with the same length as the number of rows in X
    """
    
    x_weight = 0
    for i in range(len(...)):
        x_weight = x_weight + params[i] * X.iloc[:,i] 
    p = np.exp(x_weight) / (1+np.exp(x_weight))
    
    return p.values

In [ ]:
# check dimensions of output vector and take a look at the output 
assert(len(prob_logit(logit_fit.params, X)) == X.shape[0])

prob_logit(logit_fit.params,X)

**Question 1.5 (1pt)** The `assert` block above checked the dimensions of your output. In the cell below, check that all the values in your output vector `p` are between 0 and 1 (inclusive) - i.e. we want to make sure that we're calculating probabilities! If you find that your values are not all greater than or equal to 0 and less than or equal to 1, double check the calculation in `prob_logit()`.

In [ ]:
# YOUR CODE HERE
p = prob_logit(...)
print((p >= 0).all())
print((p <= 1).all())

**Question 1.6 (1pt)** Now that we have our probabilities, we can plot our predictor variable (`WSR_PK`) vs our response variable (`CLASS`) with a curve representing the probability of an ozone day at each value of `WSR_PK` overlayed. Create that plot below - you will end up with a plot that looks like the one in 1.1, with a curve overlayed.

Note that to avoid a plot where `matplotlib` just connects the dots and you have lines all over the place, you'll have to pass a *sorted* version of `WSR_PK` to your function `prob_logit()`. Below, we've defined a variable `X_plot` that sorts the `WSR_PK` values that you can use as an input (assuming that you wrote `prob_logit()` to take a matrix `X` as input that contains features and the intercept; if you wrote `prob_logit()` to take a matrix without an intercept, you'll have to modify `X_plot`).

In [ ]:
X_plot = df[['WSR_PK', 'intercept']].sort_values(by = 'WSR_PK')

# create a scatter plot of our dependent versus independent variable observations 
plt.scatter(...)

# overlay a plot of the curve showing our predicted probability of an ozone day for a given value of WSR_PK
y = prob_logit(...)
plt.plot(...)

**Question 1.7 (1pt)** Visually, at what values of `WSR_PK` is our model pretty accurate? At what values of `WSR_PK` does it lose accuracy?

*YOUR ANSWER HERE*

Final note: Although we used only one feature for our regression, logistic regression also works with multiple features and is a powerful tool for classification. 


----

### Section 2: Intro to Decision Trees

To prepare you for the homework, let's set up a decision tree to predict ozone days using the same two features we used to train our KNN model in Section 2. We will make use of scikit-learn's `DecisionTreeClassifier`. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import tree

In [ ]:
from sklearn.model_selection import train_test_split

**Question 2.1 (1pt)** Instantiate a `DecisionTreeClassifer` model and call it `O3tree`. Set a `random_state` of 10. Fit the model using the training data, and score it using the training and test data. Assign the scores to the variables `train_score` and `val_score` respectively. 

In [ ]:
# Replace ellipses with your code
X_train, X_test, y_train, y_test = train_test_split(df[['WSR_PK', 'T_PK']], df.Class, 
                                                    test_size = 0.2, random_state=2020)

O3tree = DecisionTreeClassifier(random_state=10)
O3tree.fit(...)

print("Number of features: {}".format(...))
print("Number of nodes (internal and terminal): {}".format(...), "\n")

train_score = O3tree.score(...)
val_score = O3tree.score(...)

print('Train Score: ', train_score)
print('Validation Score: ', val_score)

What do these outputs represent? We can copy the code and visualize the tree on [Webgraphviz](http://webgraphviz.com). By running the following cell, you'll see a pretty long output -- follow the link and copy and paste the output to get a visualization of the decision tree we fit!

In [ ]:
import graphviz
print(tree.export_graphviz(O3tree, feature_names=X_train.columns))

**Question 2.2 (2pts)** You'll notice that each box in the output tree has four rows of information. What does each line in a given box in the Webgraphviz visualization represent?

***YOUR ANSWER HERE*** <br>
First line:

Second line:

Third line:

Fourth Line:

----

### Optional – Section 3: k-Nearest Neighbors for Classification

In Homework 5, we used the KNN algortithm for regression -- we predicted PM2.5 levels based on the average of the surrounding k measurements. This time around, we'll use the ozone dataset and KNN to classify whether a day is "normal" or an ozone day. Unlike our logistic regression approach, we'll be working with *two* features, namely, the peak wind speed (`WSR_PK`) and the peak temperature (`T_PK`), instead of just one feature.

Run the following cell to a see a scatter plot of the data.

In [ ]:
plt.figure(figsize=(10, 7))
plt.ylabel('Peak Temperature')
plt.xlabel('Peak Wind Speed')
for i in range(df.shape[0]):
    if df.Class[i] == 0: # if it's a normal day.
        pltcolor = 'b'
    else:
        pltcolor = 'r'
    plt.scatter(df.WSR_PK[i], df.T_PK[i], c=pltcolor)
plt.legend(['Ozone Day', 'Normal Day']);

Using KNN doesn't seem like a bad idea -- there's only a few crossovers and the possible decision boundary doesn't look too messy. 

Instead of coding the KNN algorithm from scratch like we did in Homework 5, we'll make use of scikit-learn's `KNeighborsClassifier`. Check out the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) to learn about the arguments you can tweak.

**Question 3.1 (2pts)** Split the data into training and tests sets using `train_test_split`, with `test_size = 0.2` and `random_state = 2021`. Then, instantiate a scikit-learn KNN model and fit the model with `WSR_PK` and `T_PK`. Set `n_neighbors` to 4. Then, choose a value for peak wind speed and peak temperature and use `.predict()` to determine the ozone class at those values. Is the class what you expect it to be?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Replace ellipses with your code
X_train, X_test, y_train, y_test = train_test_split(...)

knn = KNeighborsClassifier(...)
knn.fit(...)

In [ ]:
# Choose a value for WSR_PK and a value for T_PK
knn.predict([[...]]) 

**Question 3.2 (1pt)** In a couple sentences, explain in your own words how KNN works for classification problems. You can use formulas if it helps you explain or understand the method. How does KNN decide if a given wind speed and temperature corresponds to an ozone day?

*YOUR ANSWER HERE*

---

Now that we have our classifier fitted, let's test out some values of K. Before we do so, run the cell below, which defines a function that plots the decision boundary for a classifier when given a number of neighbors.

In [ ]:
def plot_boundary(model, X, y, n_neighbors):
    cmap_light = ListedColormap(['#AAAAFF', '#FFAAAA'])
    cmap_bold = ListedColormap(['#0000FF', '#FF0000'])
    h = .02
    
    x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
    y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(8, 7))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xlabel('Peak Wind Speed')
    plt.ylabel('Peak Temperature')
    plt.title("Ozone/Normal Day Classification (k = %i)"
              % (n_neighbors))
    
    plt.show()

**Question 3.3 (3pts)** Plot three decision boundaries, using a small value for K, a large value for K, and one somewhere in between. Use `.fit()` to train the model on the training data, use `plot_boundary()` to produce a plot, and use `.score()` to get the score of the model on the test data - i.e. the mean accuracy, or the proportion of test data points that were accurately classified. Make sure to show the plot and the score for each value of K.

In [ ]:
# Try a small value of k
knn1 = KNeighborsClassifier(n_neighbors = ...)
knn1.fit(...)
plot_boundary(...)
print(knn1.score(...))

In [ ]:
# Repeat for a large value of k
...

In [ ]:
# Repeat for  medium value of k
...

**Question 3.4 (1pt)** Now that we have a a few plots of various decision boundaries, what are some problems with using small or large values for K? Reference your plots in your answer.

----

## Submission

Congrats, you're done with Lab 9! 

In order to turn in this assignment, go to the toolbar and click **File** -> **Download as** -> **.pdf** and **.ipynb**. Submit the files through bCourses.

----